In [ ]:
import cv2
import numpy as np

In [ ]:
def read_video_into_numpy(filename, framecap=-1):
  cap = cv2.VideoCapture(filename)
  frames = []

  # set a frame cap
  framectr = 0

  # Check if camera opened successfully
  if (cap.isOpened()== False):
    print("Error opening video stream or file")
    return
  while cap.isOpened() and (framectr < framecap or framecap == -1):
    ret, frame = cap.read()
    framectr+=1

    # for some reason the ret is false but the frame still reads.
    # probably because avi format
    # we also can't convert from BGR to RGB but we can
    # either convert each frame
    # or just use the middle channel since it's green anyway so it's fine.

    #print(frame)
    # if frame is read correctly ret is True
    # if not ret:
    #   print("Can't receive frame (stream end?). Exiting ...")
      #break

    # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # frame = cv2.resize(frame, (16, 16))
    if frame is not None:
      # print(3)
      frame = cv2.resize(frame, (16, 16))
      frames.append(frame)
    else:
      break

  cap.release()
  video = np.stack(frames, axis=0)#converts to numpy array(T,H,W,C)
  # video = np.transpose(video, (1,2,3,0))#(T,H,W,C)->(H,W,C,T)
  return np.array(frames)

In [ ]:
video_data = read_video_into_numpy('Bad_Apple.mp4')

In [ ]:
video_data.shape

(6572, 16, 16, 3)

In [ ]:
# we can take around 10 fps (take every 6th frame, and then restrict duration to like 30s instead of 3:30, so another 7x downsample)

In [ ]:
# video_data = video_data[::6]
# video_data = video_data[:50]

In [ ]:
# Zero out entries with value 255
video_data[video_data <= 30] = 0
video_data[video_data > 30] = 255

# Print the modified array
print(video_data.max())

255


In [ ]:
video_data = video_data[..., [1, 0, 2]] # RGB to GRB

In [ ]:
def format_string(nparr):
  # Convert numpy array to string
  array_str = str(nparr)

  # Replace square brackets with curly brackets
  formatted_array_str = array_str.replace("[", "{").replace("]", "}")
  formatted_array_str = formatted_array_str.replace("\n", "").replace(" ", "").replace("000", "0, 0, 0").replace("255255255", "255, 255, 255").replace("}{", "}, {")
  formatted_array_str = formatted_array_str.replace("{255, 255, 255}","1").replace("{0, 0, 0}", "0")
  return formatted_array_str

In [ ]:
with open('bad_apple.c', 'w') as file:
  # Write the string to the file
  file.write("{")
  for fr in range(video_data.shape[0]):
    # with open('bad_apple_first50f_10fps.txt', 'w') as file:
    # Write the string to the file
    file.write(format_string(video_data[fr]))
    if fr < video_data.shape[0]-1:
      file.write(", ")

# with open('bad_apple_first50f_10fps.txt', 'w') as file:
          # Write the string to the file
  file.write("};")

In [ ]:
# Frequencies of notes in the third octave (C3 to B3)
base_notes = np.array([16.35, 17.32, 18.35, 19.45, 20.60, 21.83,
                       23.13, 24.50, 25.96, 27.50, 29.14, 30.87])


octave_34 = np.hstack((base_notes*8, base_notes*16))[:, None]

In [ ]:
base_notes*16

array([261.6 , 277.12, 293.6 , 311.2 , 329.6 , 349.28, 370.08, 392.  ,
       415.36, 440.  , 466.24, 493.92])

In [ ]:
octave_34

array([[130.8 ],
       [138.56],
       [146.8 ],
       [155.6 ],
       [164.8 ],
       [174.64],
       [185.04],
       [196.  ],
       [207.68],
       [220.  ],
       [233.12],
       [246.96],
       [261.6 ],
       [277.12],
       [293.6 ],
       [311.2 ],
       [329.6 ],
       [349.28],
       [370.08],
       [392.  ],
       [415.36],
       [440.  ],
       [466.24],
       [493.92]])

In [ ]:
minor_triads = np.array([[pitch[0], pitch[0]*32/27, pitch[0]*1.5] for pitch in octave_34])

In [ ]:
major_triads = np.array([[pitch[0], pitch[0]*81/64, pitch[0]*1.5] for pitch in octave_34])

In [ ]:
str(octave_34).replace("[", "{").replace("]", "}").replace("\n", ",")

'{{130.81}, {138.59}, {146.83}, {155.56}, {164.81}, {174.61}, {185.  }, {196.  }, {207.65}, {220.  }, {233.08}, {246.94}, {261.63}, {277.18}, {293.66}, {311.13}, {329.63}, {349.23}, {369.99}, {392.  }, {415.3 }, {440.  }, {466.16}, {493.88}}'

In [ ]:
minor_triads

array([[130.8       , 155.02222222, 196.2       ],
       [138.56      , 164.21925926, 207.84      ],
       [146.8       , 173.98518519, 220.2       ],
       [155.6       , 184.41481481, 233.4       ],
       [164.8       , 195.31851852, 247.2       ],
       [174.64      , 206.98074074, 261.96      ],
       [185.04      , 219.30666667, 277.56      ],
       [196.        , 232.2962963 , 294.        ],
       [207.68      , 246.13925926, 311.52      ],
       [220.        , 260.74074074, 330.        ],
       [233.12      , 276.29037037, 349.68      ],
       [246.96      , 292.69333333, 370.44      ],
       [261.6       , 310.04444444, 392.4       ],
       [277.12      , 328.43851852, 415.68      ],
       [293.6       , 347.97037037, 440.4       ],
       [311.2       , 368.82962963, 466.8       ],
       [329.6       , 390.63703704, 494.4       ],
       [349.28      , 413.96148148, 523.92      ],
       [370.08      , 438.61333333, 555.12      ],
       [392.        , 464.59259

In [ ]:
str(minor_triads).replace("      ", ",").replace("[", "{").replace("]", "}").replace("\n", ",")

'{{130.81, 155.03407407 196.215     }, {138.59, 164.25481481 207.885     }, {146.83, 174.02074074 220.245     }, {155.56, 184.36740741 233.34,}, {164.81, 195.33037037 247.215     }, {174.61, 206.94518519 261.915     }, {185.,   219.25925926 277.5, }, {196.,   232.2962963  294.,  }, {207.65, 246.1037037  311.475     }, {220.,   260.74074074 330.,  }, {233.08, 276.24296296 349.62,}, {246.94, 292.66962963 370.41,}, {261.63, 310.08, 392.445     }, {277.18, 328.50962963 415.77,}, {293.66, 348.04148148 440.49,}, {311.13, 368.74666667 466.695     }, {329.63, 390.67259259 494.445     }, {349.23, 413.90222222 523.845     }, {369.99, 438.50666667 554.985     }, {392.,   464.59259259 588.,  }, {415.3,  492.20740741 622.95,}, {440.,   521.48148148 660.,  }, {466.16, 552.48592593 699.24,}, {493.88, 585.33925926 740.82,}}'

In [ ]:
major_triads

array([[130.8    , 165.54375, 196.2    ],
       [138.56   , 175.365  , 207.84   ],
       [146.8    , 185.79375, 220.2    ],
       [155.6    , 196.93125, 233.4    ],
       [164.8    , 208.575  , 247.2    ],
       [174.64   , 221.02875, 261.96   ],
       [185.04   , 234.19125, 277.56   ],
       [196.     , 248.0625 , 294.     ],
       [207.68   , 262.845  , 311.52   ],
       [220.     , 278.4375 , 330.     ],
       [233.12   , 295.0425 , 349.68   ],
       [246.96   , 312.55875, 370.44   ],
       [261.6    , 331.0875 , 392.4    ],
       [277.12   , 350.73   , 415.68   ],
       [293.6    , 371.5875 , 440.4    ],
       [311.2    , 393.8625 , 466.8    ],
       [329.6    , 417.15   , 494.4    ],
       [349.28   , 442.0575 , 523.92   ],
       [370.08   , 468.3825 , 555.12   ],
       [392.     , 496.125  , 588.     ],
       [415.36   , 525.69   , 623.04   ],
       [440.     , 556.875  , 660.     ],
       [466.24   , 590.085  , 699.36   ],
       [493.92   , 625.1175 , 740.

In [ ]:
import imageio

In [ ]:

imageio.mimsave('bad_apple_bin.gif', video_data, fps=60)
